In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

D:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
auction = []
close_auction = [] 
name_lot = []
year_lot = []
material_lot = []
quality_lot = []
last_bet = []


for number_auction in range(1700,1702):
    req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction))
    soup = BeautifulSoup(req.text, "html.parser")

    if re.search(r'(VIP)', soup.find('h1').text) is not None or re.search(r'(Standart)', soup.find('h1').text) is not None:
        date_close = soup.find('h1').find('span').text[8:18]
        print(date_close)
        
        for i in soup.find('ul').findAll('li')[::-1]:
            last_page = i.text
            break

        for page in range(1,int(last_page)+1):
            page_req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction)+"?page="+str(page))
            page_soup = BeautifulSoup(page_req.text, "html.parser")
            
            for i in page_soup.findAll('table')[4].findAll('tr')[3:]:
                if i.findAll('td')[1].find('a') is not None:
                    auction.append("https://www.wolmar.ru/auction/"+str(number_auction))
                    close_auction.append(date_close)  
                    name_lot.append(i.findAll('td')[1].find('a').text)
                    year_lot.append(i.findAll('td')[2].text)
                    material_lot.append(i.findAll('td')[4].text)
                    quality_lot.append(i.findAll('td')[5].find('nobr').text)
                    last_bet.append(i.findAll('td')[8].text)

29.12.2021


In [3]:
df = pd.DataFrame({'auction': auction,
                   'date_close_auction':close_auction,
                   'name':name_lot,
                   'year': year_lot,
                   'material':material_lot,
                   'quality': quality_lot,
                   'last_bet':last_bet})
df.to_csv('history_auction.csv',
          index=False)